## Tariff Analysis Example

This is an example of how to apply tariffs on load profiles, visualise the results, and do further analysis. To get the notebook and asscociated python files refer to https://github.com/UNSW-CEEM/Bill_Calculator

Prepared by: Navid Haghdadi , March 2019
Updated Nov 2019

In [1]:
# Importing the libraries we want:
import pandas as pd
import numpy as np
from Bill_Calc import bill_calculator as calc
import json
import io
from datetime import datetime
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display
import ipywidgets as widgets
import requests
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import plotly
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

np.warnings.filterwarnings('ignore')

## 1. Reading the load profile
Put the path of load profile as below

In [2]:
LoadProfiles=pd.read_csv('SampleLoadProfile.csv')
LoadProfiles['Datetime']=pd.to_datetime(LoadProfiles['Datetime'],format='%d/%m/%Y %H:%M')

In [3]:
def DataPlot1(HomeNo):
    data = [{"x":LoadProfiles['Datetime'], "y":LoadProfiles.iloc[:,HomeNo]}]
    layout = go.Layout(title='Home number: '+str(HomeNo))
    iplot({"data":data,"layout":layout})

In [4]:
w=interactive(DataPlot1, HomeNo=list(range(1, LoadProfiles.shape[1]-1)))
display(w)

interactive(children=(Dropdown(description='HomeNo', options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1…

In [5]:
# I've prepared a sample network load as well (aggregation of 4000 homes as a proxy for network load)
NetworkLoad=pd.read_csv('NetworkLoad.csv')
NetworkLoad['Datetime']=pd.to_datetime(NetworkLoad['Datetime'],format='%d/%m/%Y %H:%M')
data = [{"x":NetworkLoad['Datetime'], "y":NetworkLoad['kWh']}]
layout = go.Layout(showlegend=True, title='Network load')
iplot({"data":data,"layout":layout})

In [6]:
Allsum=LoadProfiles.sum()/365 # kWh/day

data=[go.Histogram(x=Allsum)]
layout = go.Layout(showlegend=True, title='kWh/day')

iplot({"data":data,"layout":layout})

In [7]:
# Let's compare the peak value of each home to the value of each home at the time of network peak (coincident peak):
Coincident_Peak_Time=NetworkLoad.loc[NetworkLoad['kWh'].idxmax(),'Datetime'] 
Coincident_Peak_Value=NetworkLoad.loc[NetworkLoad['kWh'].idxmax(),'kWh']
display('Coincident Peak Time = '+ str(Coincident_Peak_Time))
display('Coincident Peak Value = '+ str(Coincident_Peak_Value) +' kWh')

'Coincident Peak Time = 2013-01-18 18:00:00'

'Coincident Peak Value = 7341.719 kWh'

In [8]:

data =[go.Scatter(x=LoadProfiles.iloc[NetworkLoad['kWh'].idxmax(),1:].values, y= LoadProfiles.iloc[:,1:].max(),name='Total Bill ($)', mode='markers')]
   
layout = go.Layout(title='Comparison of home peak and network peak',xaxis=dict(title='Coincident peak (kWh)'),yaxis=dict(title='Home peak (kWh)'))
fig = go.Figure(data, layout)
iplot(fig)

### Applying Tariffs
First you need to select the tariff. You can directly access the CEEM tariff API http://api.ceem.org.au/elec-tariffs/retail for **retail tariffs** or http://api.ceem.org.au/elec-tariffs/network for **network tariff**. Please note it may take few seconds to get the data.

Alternatively you can use the offline json file in the repositoy:
`AllTariffs_network.json` or `AllTariffs_retail.json`

You can update the list of tariff (get the latest version from API) as described below.

**Option 1: Get it from API**

In [12]:
#  Network
all_tariffs = requests.get('http://api.ceem.org.au/electricity-tariffs/network')
all_tariffs_list= all_tariffs.json()
all_tariffs_list = all_tariffs_list[0]['Tariffs']
#  Retail
all_tariffs = requests.get('http://api.ceem.org.au/electricity-tariffs/retail')
all_tariffs_list= all_tariffs.json()
all_tariffs_list = all_tariffs_list[0]['Tariffs']

**Option 2: Get it from local json file (network or retail)**

In [2]:
#  Network
# with open('AllTariffs_network.json') as f:
#      all_tariffs_list = json.load(f)
# Retail        
# with open('AllTariffs_retail.json') as f:
#      all_tariffs_list = json.load(f)



**Update the local json file from the API:**

In [ ]:
# # Network tariff
# all_tariffs = requests.get('http://api.ceem.org.au/elec-tariffs/network')
# all_tariffs_list= all_tariffs.json()
# with io.open('AllTariffs_network.json', 'w', encoding='utf8') as outfile:
#     json.dump(all_tariffs_list, outfile)
    
# # Retail tariff
# all_tariffs = requests.get('http://api.ceem.org.au/elec-tariffs/retail')
# all_tariffs_list= all_tariffs.json()
# with io.open('AllTariffs_retail.json', 'w', encoding='utf8') as outfile:
#     json.dump(all_tariffs_list, outfile)

## Select the tariff ##

In [13]:
TariffList=[]
for i in range(len(all_tariffs_list)):
#     display(all_tariffs_list[i]['Name'])
    TariffList.append(all_tariffs_list[i]['Name'])
    
def SelTariff(Tariff):
    for i in range(len(all_tariffs_list)):
        if all_tariffs_list[i]['Name'] == Tariff:
            selected_tariff = all_tariffs_list[i] 
    display(selected_tariff)
    # print(json.dumps(selected_tariff, indent=2))

w=interactive(SelTariff, Tariff=TariffList)
display(w)
# SelTariff('Origin Flat Rate NSW (Endeavour area)')  



interactive(children=(Dropdown(description='Tariff', options=('Next Business Energy Block daily Evoenergy area…

## Apply the tariff

In [27]:
for i in range(len(all_tariffs_list)):
        if all_tariffs_list[i]['Name'] == w.kwargs['Tariff']:
            tariff = all_tariffs_list[i] 
            
load_profile = LoadProfiles.copy().set_index('Datetime')
Results = calc(load_profile, tariff)

Results['LoadInfo'].head()
Results['Retailer'].head()
# Results['DUOS'].head()

,Annual_kWh,Annual_kWh_exp
8143479,1644.9765,0.0
8143507,9178.1945,0.0
8143533,6027.4625,0.0
8143563,5868.7295,0.0
8143599,8123.3670,0.0


,Charge_FiT_Rebate,Charge_Daily,Charge_BlockDaily,Bill
8143479,0.0,410.7345,538.499507,949.234007
8143507,0.0,410.7345,3004.573752,3415.308252
8143533,0.0,410.7345,1973.150124,2383.884624
8143563,0.0,410.7345,1921.187289,2331.921789
8143599,0.0,410.7345,2659.265421,3069.999921


## Download the result
Select columns you want to export and then click Export to CSV. The CSV file will be saved in the same folder as you have this notebook with the current date and time in the name of it. 

In [34]:
Results_ofinterest = Results['LoadInfo']
# or
Results_ofinterest = Results['Retailer']

W2=widgets.SelectMultiple(options=Results_ofinterest.columns,rows=10,description='Columns',disabled=False)
display(W2)
W3=widgets.Button(description='Export to CSV',disabled=False,button_style='info',tooltip='Export to CSV',icon='check')
display(W3)

def on_button_clicked(b):
    Results2 = Results_ofinterest[Results_ofinterest.columns.intersection(W2.value)]
    Results2.to_csv('Results' + datetime.now().strftime("%Y%m%d_%H%M" +'.csv'))
W3.on_click(on_button_clicked)

SelectMultiple(description='Columns', options=('Charge_FiT_Rebate', 'Charge_Daily', 'Charge_BlockDaily', 'Bill…

Button(button_style='info', description='Export to CSV', icon='check', style=ButtonStyle(), tooltip='Export to…

## Plot the results
Click on the legend of the columns you want to see

In [ ]:
# # Plotting the results:
# def PlotResult(Results_ofinterest):
#     data =[]
#     # data
#     for i in Results_ofinterest.columns:
#         data.append(go.Scatter(x= Results['Annual_kWh'], y= Results[i], name=i, mode='markers', visible = "legendonly"))
#     layout = go.Layout(title='Results', xaxis=dict(title='kWh'))
#     fig = go.Figure(data, layout)
#     iplot(fig)
    
# PlotResult(Results)

In [35]:
# Now let's bring solar to the game!

SolarProf=pd.read_csv('100SolarProf.csv')
SolarProf['Datetime']=pd.to_datetime(SolarProf['Datetime'],format='%d/%m/%Y %H:%M')


# W3=widgets.ToggleButtons(options=['Solar Generation', 'kWh/Day', 'Average load and solar'],
#     description='Chart type:', disabled=False, button_style='', # 'success', 'info', 'warning', 'danger' or ''
#     tooltips=['Solar Generation', 'kWh/Day', 'Average load and solar'])
# display(W3)

# def on_click(change):
#     if W3.value == 'Solar Generation':
data =[go.Scatter(x=SolarProf['Datetime'], y= np.nanmean(SolarProf.iloc[:,1:].values,axis=1),name='Total Bill ($)', mode='lines')]
layout = go.Layout(title='Average of solar homes',xaxis=dict(title='time'),yaxis=dict(title='Solar Generation (kWh)'))
fig=go.Figure(data, layout)
iplot(fig)
        
#     elif W3.value == 'kWh/Day':
AllsumPV=SolarProf.sum()/365 # kWh/day
data=[go.Histogram(x=AllsumPV)]
layout = go.Layout(showlegend=True, title='kWh/day')
iplot({"data":data,"layout":layout})
#     elif W3.value == 'Average load and solar':
data =[go.Scatter(x=SolarProf['Datetime'], y= np.nanmean(load_profile.iloc[:,1:].values,axis=1),name='Load', mode='lines'),
go.Scatter(x=SolarProf['Datetime'], y= np.nanmean(NetLoad.iloc[:,1:].values,axis=1),name='Netload', mode='lines'),
go.Scatter(x=SolarProf['Datetime'], y= np.nanmean(SolarProf.iloc[:,1:].values,axis=1),name='Solar', mode='lines')]
layout = go.Layout(title='Time',xaxis=dict(title='time'),yaxis=dict(title='Load and Solar'))
fig=go.Figure(data, layout)
iplot(fig)
        
# W3.observe(on_click, 'value')


NameError: name 'NetLoad' is not defined

In [ ]:
#  Now we can subtract the solar profile from load profile to find out the net load 
# (we assume these 100 solar profiles belong to those 100 homes)

CombinedLoad=LoadProfiles.merge(SolarProf,on='Datetime')
NetLoad=pd.concat([CombinedLoad['Datetime'],pd.DataFrame(data=CombinedLoad[CombinedLoad.columns[1:101]].values-CombinedLoad[CombinedLoad.columns[101:]].values)],axis=1)

# Take a look at the load:
data = [{"x":NetLoad['Datetime'], "y":NetLoad[col]} for col in NetLoad.columns[1:10]]
layout = go.Layout(showlegend=True, title='First 10 homes (Net load)')
iplot({"data":data,"layout":layout})

In [ ]:
#  Now let's calculate the bill again 
Net_Load = NetLoad.copy().set_index('Datetime')
Results_FR_Sol = calc(Net_Load, tariff)
PlotResult(Results_FR_Sol)

In [ ]:
# Now let's double the capacity of PV systems:
NetLoad_DoublePV=pd.concat([CombinedLoad['Datetime'],pd.DataFrame(data=CombinedLoad[CombinedLoad.columns[1:101]].values-2*CombinedLoad[CombinedLoad.columns[101:]].values)],axis=1)
Net_Load_DoublePV = NetLoad_DoublePV.copy().set_index('Datetime')

Results_FR_Sol_2 = calc(Net_Load_DoublePV, tariff)
PlotResult(Results_FR_Sol_2)

In [ ]:
data =[go.Scatter(x= Results_FR_Sol_2['Annual_kWh'], y= Results['Bill'],name='Total Bill ($) - without solar', mode='markers'),
      go.Scatter(x= Results_FR_Sol_2['Annual_kWh'], y= Results_FR_Sol['Bill'],name='Total Bill ($) - with solar', mode='markers'),
      go.Scatter(x= Results_FR_Sol_2['Annual_kWh'], y= Results_FR_Sol_2['Bill'],name='Total Bill ($) - with double solar', mode='markers')]
layout = go.Layout(title='Comparison of bills in three solar scenarios',xaxis=dict(title='kWh'),yaxis=dict(title='Bill'))
fig = go.Figure(data, layout)
iplot(fig)

In [ ]:
#  How about the TOU tariff?

Results_TOU_Sol = calc(Net_Load, tariff_TOU)
Results_TOU_Sol_2 = calc(Net_Load_DoublePV, tariff_TOU)

data =[go.Scatter(x= Results_TOU['Annual_kWh'], y= Results_TOU['Bill'],name='Total Bill ($) - without solar', mode='markers'),
      go.Scatter(x= Results_TOU_Sol['Annual_kWh'], y= Results_TOU_Sol['Bill'],name='Total Bill ($) - with solar', mode='markers'),
      go.Scatter(x= Results_TOU_Sol_2['Annual_kWh'], y= Results_TOU_Sol_2['Bill'],name='Total Bill ($) - with double solar', mode='markers')]
layout = go.Layout(title='Comparison of bills in three solar scenarios for TOU tariffs',xaxis=dict(title='kWh'),yaxis=dict(title='Bill'))
fig = go.Figure(data, layout)
iplot(fig)


In [ ]:
#  Now let's have a look at the appliance level data
# A sample load profile from Smart Grid Smart City trial
# Reading the load profile

LoadProfile_App=pd.read_csv('ApplianceData.csv')
LoadProfile_App['TimeStamp']=pd.to_datetime(LoadProfile_App['TimeStamp'],format='%d/%m/%Y %H:%M')
# Take a look at the load:
data = [dict(x=LoadProfile_App['TimeStamp'], y=LoadProfile_App['Aircon'],stackgroup='one',name='Aircon'),
       dict(x=LoadProfile_App['TimeStamp'], y=LoadProfile_App['Computer'],stackgroup='one',name='Computer'),
        dict(x=LoadProfile_App['TimeStamp'], y=LoadProfile_App['B1'],stackgroup='one',name='B1'),
       dict(x=LoadProfile_App['TimeStamp'], y=LoadProfile_App['Dishwasher'],stackgroup='one',name='Dishwasher'),
       dict(x=LoadProfile_App['TimeStamp'], y=LoadProfile_App['Microwave'],stackgroup='one',name='Microwave'),
       dict(x=LoadProfile_App['TimeStamp'], y=LoadProfile_App['Oven'],stackgroup='one',name='Oven'),
       dict(x=LoadProfile_App['TimeStamp'], y=LoadProfile_App['TV'],stackgroup='one',name='TV'),
       dict(x=LoadProfile_App['TimeStamp'], y=LoadProfile_App['TV2'],stackgroup='one',name='TV2'),
        dict(x=LoadProfile_App['TimeStamp'], y=LoadProfile_App['TV3'],stackgroup='one',name='TV3'),
        dict(x=LoadProfile_App['TimeStamp'], y=LoadProfile_App['Washing'],stackgroup='one',name='Washing'),
         dict(x=LoadProfile_App['TimeStamp'], y=LoadProfile_App['HotPlates'],stackgroup='one',name='HotPlates')
       ]
layout = go.Layout(title='Appliance Level Data')
iplot({"data":data,"layout":layout})


In [ ]:
LoadProfile_App=LoadProfile_App.rename(columns={'TimeStamp':'Datetime'})
LoadProfile_App2=LoadProfile_App.set_index('Datetime')
Results_AppData = calc(LoadProfile_App2, tariff)
display(Results_AppData)